# **KW AI Hackathon - 컴퓨터 비전 AI 경진대회**



## - 팀 04 씽크빅데이터 - 

데이터의 크기가 커서 다양한 실험이 어려웠던 대회였습니다. 

TPU 사용을 고려했으나, ImageDataGenerator와의 충돌 문제를 해결하지 못하고, 

결국 GPU로 학습을 진행하여 시간 제약이 크게 다가온 점과 생각보다 점수 개선이 더디게 된 점이 아쉬웠지만. 본 대회를 통해 Multi-Label 문제도, EfficientNet_V2모델도 처음 사용해 보면서 한 층 더 성장할 수 있었던 것 같습니다.

남은 기간 동안에도 좋은 성과 있으시길 바라겠습니다!


Framework는 Keras, 환경은 Google Colab pro+(GPU) 입니다.


# Version

In [ ]:
import sys
import tensorflow as tf
import keras
import sklearn
import numpy
import pandas
sys.version

!python --version
print('TensorFlow ' +  tf.__version__)
print('Keras ' + keras.__version__)
print('Sklearn ' + sklearn.__version__)
print('Numpy ' + numpy.__version__)
print('Pandas ' + pandas.__version__)

Python 3.7.14
TensorFlow 2.8.2
Keras 2.8.0
Sklearn 1.0.2
Numpy 1.21.6
Pandas 1.3.5


# Google drive에서 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import output
# !cp 파일1 파일2 # 파일1을 파일2로 복사 붙여넣기
!cp "/content/drive/MyDrive/kw_cv_comp/dataset/comp_data/235697_제 2회 컴퓨터 비전 학습 경진대회_data.zip" "data_2.zip" # 데이터셋 파일 경로 지정
# 위 파일 경로를 google drive 중 대회 데이터셋에 해당하는 경로로 지정하시면 colab 환경에서 그대로 작동합니다.

# data_2.zip을 현재 디렉터리에 압축해제
!unzip "data_2.zip"

Archive:  data_2.zip
  inflating: dirty_mnist_2nd.zip     
  inflating: dirty_mnist_2nd_answer.csv  
  inflating: sample_submission.csv   
  inflating: test_dirty_mnist_2nd.zip  


In [ ]:
from google.colab import output
# 현재 디렉터리에 dirty_mnist라는 폴더 생성
!mkdir "./dirty_mnist"
#dirty_mnist.zip라는 zip파일을 dirty_mnist라는 폴더에 압축 풀기
!unzip "dirty_mnist_2nd.zip" -d "./dirty_mnist/"
# 현재 디렉터리에 test_dirty_mnist라는 폴더 생성
!mkdir "./test_dirty_mnist"
#test_dirty_mnist.zip라는 zip파일을 test_dirty_mnist라는 폴더에 압축 풀기
!unzip "test_dirty_mnist_2nd.zip" -d "./test_dirty_mnist/"
# 출력 결과 지우기
output.clear()

# Library Import

In [ ]:
from google.colab import output
from keras import backend as K 

import os
import zipfile

import pandas as pd
import numpy as np

import cv2
from tqdm import tqdm

import random as python_random # 시드값 고정
seed_num = 42
np.random.seed(seed_num)
python_random.seed(seed_num)
tf.random.set_seed(seed_num)


#  Dataset

In [ ]:
train = pd.read_csv('/content/dirty_mnist_2nd_answer.csv')
test = pd.read_csv('/content/sample_submission.csv')
sub = pd.read_csv('/content/sample_submission.csv')

In [ ]:
col_list = np.array(train.iloc[:,1:].columns) # a,b,c,d,e.. 등의 column들의 이름을 1차원 넘파이로 표현
num_list = np.array(train.iloc[:,1:]) # 각 데이터가 갖는 label에 대한 값들을 binary 형식 1차원 넘파이로 표현
multi_list = num_list * col_list # 모델의 target data로 사용하기 위해 각 이미지에 대한 label 값을 1차원 넘파이로 저장
label_pd = pd.DataFrame(multi_list).apply(lambda x : ' '.join(x),axis=1) 
label_pd = pd.Series(label_pd) 
train['label'] = label_pd # train data에 각 index에 해당하는 1차원 넘파이로 표시된 label 값을 column으로 추가
train['label'] = train['label'].apply(lambda x : x.split()) # label 값 중 1이 아닌 값들이 ''로 나타나므로 따로 처리
train

,index,a,b,c,d,e,f,g,h,i,...,r,s,t,u,v,w,x,y,z,label
0,0,1,1,0,1,0,1,0,0,0,...,1,1,0,1,0,0,1,1,1,"[a, b, d, f, k, l, o, p, r, s, u, x, y, z]"
1,1,1,0,0,1,0,1,0,1,0,...,1,0,1,0,0,0,0,1,1,"[a, d, f, h, j, l, p, r, t, y, z]"
2,2,0,0,0,0,0,0,0,0,1,...,0,1,1,1,0,1,1,1,0,"[i, j, n, p, s, t, u, w, x, y]"
3,3,0,0,1,0,0,0,1,1,0,...,0,1,1,0,1,1,0,1,0,"[c, g, h, l, p, q, s, t, v, w, y]"
4,4,0,1,0,1,0,1,0,1,1,...,1,0,0,0,1,0,1,0,0,"[b, d, f, h, i, k, m, p, r, v, x]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,0,1,1,0,0,0,0,1,0,...,0,0,1,0,0,0,1,1,0,"[b, c, h, k, l, n, o, t, x, y]"
49996,49996,0,1,0,1,0,1,1,1,0,...,1,1,1,0,1,0,0,0,1,"[b, d, f, g, h, j, o, r, s, t, v, z]"
49997,49997,0,1,0,0,1,1,1,1,0,...,0,0,0,0,1,1,1,0,0,"[b, e, f, g, h, k, l, q, v, w, x]"
49998,49998,0,1,1,1,0,0,1,1,0,...,1,1,1,0,0,0,1,0,0,"[b, c, d, g, h, j, k, l, m, n, r, s, t, x]"


In [ ]:
train['path'] = '/content/dirty_mnist/' + train['index'].apply(lambda x : str(x).zfill(5)) + '.png'
train

,index,a,b,c,d,e,f,g,h,i,...,s,t,u,v,w,x,y,z,label,path
0,0,1,1,0,1,0,1,0,0,0,...,1,0,1,0,0,1,1,1,"[a, b, d, f, k, l, o, p, r, s, u, x, y, z]",/content/dirty_mnist/00000.png
1,1,1,0,0,1,0,1,0,1,0,...,0,1,0,0,0,0,1,1,"[a, d, f, h, j, l, p, r, t, y, z]",/content/dirty_mnist/00001.png
2,2,0,0,0,0,0,0,0,0,1,...,1,1,1,0,1,1,1,0,"[i, j, n, p, s, t, u, w, x, y]",/content/dirty_mnist/00002.png
3,3,0,0,1,0,0,0,1,1,0,...,1,1,0,1,1,0,1,0,"[c, g, h, l, p, q, s, t, v, w, y]",/content/dirty_mnist/00003.png
4,4,0,1,0,1,0,1,0,1,1,...,0,0,0,1,0,1,0,0,"[b, d, f, h, i, k, m, p, r, v, x]",/content/dirty_mnist/00004.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,0,1,1,0,0,0,0,1,0,...,0,1,0,0,0,1,1,0,"[b, c, h, k, l, n, o, t, x, y]",/content/dirty_mnist/49995.png
49996,49996,0,1,0,1,0,1,1,1,0,...,1,1,0,1,0,0,0,1,"[b, d, f, g, h, j, o, r, s, t, v, z]",/content/dirty_mnist/49996.png
49997,49997,0,1,0,0,1,1,1,1,0,...,0,0,0,1,1,1,0,0,"[b, e, f, g, h, k, l, q, v, w, x]",/content/dirty_mnist/49997.png
49998,49998,0,1,1,1,0,0,1,1,0,...,1,1,0,0,0,1,0,0,"[b, c, d, g, h, j, k, l, m, n, r, s, t, x]",/content/dirty_mnist/49998.png


In [ ]:
test['path'] =  '/content/test_dirty_mnist/' + test['index'].apply(lambda x : str(x).zfill(5)) + '.png'
test

,index,a,b,c,d,e,f,g,h,i,...,r,s,t,u,v,w,x,y,z,path
0,50000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,/content/test_dirty_mnist/50000.png
1,50001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,/content/test_dirty_mnist/50001.png
2,50002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,/content/test_dirty_mnist/50002.png
3,50003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,/content/test_dirty_mnist/50003.png
4,50004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,/content/test_dirty_mnist/50004.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,54995,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,/content/test_dirty_mnist/54995.png
4996,54996,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,/content/test_dirty_mnist/54996.png
4997,54997,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,/content/test_dirty_mnist/54997.png
4998,54998,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,/content/test_dirty_mnist/54998.png


In [ ]:
display(train,test) # Path 및 Target 값 구현 확인

,index,a,b,c,d,e,f,g,h,i,...,s,t,u,v,w,x,y,z,label,path
0,0,1,1,0,1,0,1,0,0,0,...,1,0,1,0,0,1,1,1,"[a, b, d, f, k, l, o, p, r, s, u, x, y, z]",/content/dirty_mnist/00000.png
1,1,1,0,0,1,0,1,0,1,0,...,0,1,0,0,0,0,1,1,"[a, d, f, h, j, l, p, r, t, y, z]",/content/dirty_mnist/00001.png
2,2,0,0,0,0,0,0,0,0,1,...,1,1,1,0,1,1,1,0,"[i, j, n, p, s, t, u, w, x, y]",/content/dirty_mnist/00002.png
3,3,0,0,1,0,0,0,1,1,0,...,1,1,0,1,1,0,1,0,"[c, g, h, l, p, q, s, t, v, w, y]",/content/dirty_mnist/00003.png
4,4,0,1,0,1,0,1,0,1,1,...,0,0,0,1,0,1,0,0,"[b, d, f, h, i, k, m, p, r, v, x]",/content/dirty_mnist/00004.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,0,1,1,0,0,0,0,1,0,...,0,1,0,0,0,1,1,0,"[b, c, h, k, l, n, o, t, x, y]",/content/dirty_mnist/49995.png
49996,49996,0,1,0,1,0,1,1,1,0,...,1,1,0,1,0,0,0,1,"[b, d, f, g, h, j, o, r, s, t, v, z]",/content/dirty_mnist/49996.png
49997,49997,0,1,0,0,1,1,1,1,0,...,0,0,0,1,1,1,0,0,"[b, e, f, g, h, k, l, q, v, w, x]",/content/dirty_mnist/49997.png
49998,49998,0,1,1,1,0,0,1,1,0,...,1,1,0,0,0,1,0,0,"[b, c, d, g, h, j, k, l, m, n, r, s, t, x]",/content/dirty_mnist/49998.png


,index,a,b,c,d,e,f,g,h,i,...,r,s,t,u,v,w,x,y,z,path
0,50000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,/content/test_dirty_mnist/50000.png
1,50001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,/content/test_dirty_mnist/50001.png
2,50002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,/content/test_dirty_mnist/50002.png
3,50003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,/content/test_dirty_mnist/50003.png
4,50004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,/content/test_dirty_mnist/50004.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,54995,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,/content/test_dirty_mnist/54995.png
4996,54996,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,/content/test_dirty_mnist/54996.png
4997,54997,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,/content/test_dirty_mnist/54997.png
4998,54998,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,/content/test_dirty_mnist/54998.png


# Data Augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import KFold
from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.applications.efficientnet_v2 import *
from tensorflow.keras.callbacks import *
import albumentations as al

In [ ]:
def rotate_image(image):
   return np.rot90(image, np.random.choice([-1, 0, 1, 2])) # 90도씩 회전하여 fill_mode가 적용되지 않는 rotation

In [ ]:
idg_rotate = ImageDataGenerator(rotation_range=90)
idg_nomal = ImageDataGenerator()

# LAB

In [ ]:
sklearn.model_selection.train_test_split

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(train, train['label'], test_size=0.9, random_state=42)
test_set_data, X_test, test_set_label, y_test = sklearn.model_selection.train_test_split(X_test, y_test, test_size=0.9, random_state=42)
train_set_data, valid_set_data, train_set_label, valid_set_label = sklearn.model_selection.train_test_split(X_train, y_train, test_size=0.25, random_state=42)

print(train_set_data.sum())
print(valid_set_data.sum())
print(test_set_data.sum())

train_set_data
valid_set_data
test_set_data



,index,a,b,c,d,e,f,g,h,i,...,s,t,u,v,w,x,y,z,label,path
8202,8202,0,0,0,0,1,1,0,1,0,...,1,1,0,0,0,0,1,0,"[e, f, h, j, l, m, o, p, q, s, t, y]",/content/dirty_mnist/08202.png
9708,9708,0,1,1,0,0,0,0,1,0,...,1,0,1,0,0,0,0,1,"[b, c, h, m, o, q, r, s, u, z]",/content/dirty_mnist/09708.png
35886,35886,1,0,0,0,1,1,0,0,1,...,1,0,0,0,1,1,0,0,"[a, e, f, i, l, m, q, s, w, x]",/content/dirty_mnist/35886.png
45881,45881,1,1,0,1,0,1,1,0,1,...,1,0,0,0,1,0,0,1,"[a, b, d, f, g, i, k, n, p, q, r, s, w, z]",/content/dirty_mnist/45881.png
21387,21387,0,0,0,0,1,1,1,0,0,...,1,0,1,0,0,1,1,1,"[e, f, g, o, q, s, u, x, y, z]",/content/dirty_mnist/21387.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3318,3318,0,1,0,1,0,0,1,0,0,...,1,1,0,0,0,0,1,1,"[b, d, g, j, p, r, s, t, y, z]",/content/dirty_mnist/03318.png
26375,26375,0,1,1,1,1,1,0,0,0,...,1,0,1,0,0,0,1,0,"[b, c, d, e, f, k, m, o, p, r, s, u, y]",/content/dirty_mnist/26375.png
44094,44094,1,0,0,0,0,0,1,0,0,...,0,1,0,1,0,0,0,0,"[a, g, l, m, n, o, p, r, t, v]",/content/dirty_mnist/44094.png
26037,26037,1,0,0,1,0,0,1,0,1,...,0,0,1,0,1,1,1,0,"[a, d, g, i, k, m, o, q, r, u, w, x, y]",/content/dirty_mnist/26037.png


In [ ]:
print(train_set_data.sum())
print(valid_set_data.sum())
print(test_set_data.sum())

index                                             93191494
a                                                     1660
b                                                     1744
c                                                     1718
d                                                     1705
e                                                     1704
f                                                     1770
g                                                     1727
h                                                     1749
i                                                     1698
j                                                     1754
k                                                     1780
l                                                     1687
m                                                     1639
n                                                     1786
o                                                     1750
p                                                     17

# Training

In [ ]:
train_gen_lab = idg_rotate.flow_from_dataframe(train_set_data, x_col='path', y_col='label', target_size=(256,256), batch_size=32)
valid_gen_lab = idg_rotate.flow_from_dataframe(valid_set_data, x_col='path', y_col='label', target_size=(256,256), batch_size=32)
test_gen_lab = idg_nomal.flow_from_dataframe(test_set_data, x_col='path', y_col='label', target_size=(256,256), batch_size=32)

Found 3750 validated image filenames belonging to 26 classes.
Found 1250 validated image filenames belonging to 26 classes.
Found 4500 validated image filenames belonging to 26 classes.


In [ ]:
!pip install tensorflow-addons
import tensorflow_addons as tfa
AdamW = tfa.optimizers.AdamW(learning_rate=0.001,weight_decay=0.00001, amsgrad=False)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 18.0 MB/s 


In [ ]:
class SAMModel(tf.keras.models.Model):
    def __init__(self, *args, rho=0.05, **kwargs):
        super().__init__( *args, **kwargs)
        self.rho = tf.constant(rho, dtype=tf.float32)

    def train_step(self,data):
        x, y = data

        # 1st step
        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)

        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        norm = tf.linalg.global_norm(gradients)
        scale = self.rho / (norm + 1e-12)
        e_w_list = []
        for v, grad in zip(trainable_vars, gradients):
            e_w = grad * scale
            v.assign_add(e_w)
            e_w_list.append(e_w)

        # 2nd step
        with tf.GradientTape() as tape:
            y_pred_adv = self(x, training=True)
            loss_adv = self.compiled_loss(y, y_pred_adv, regularization_losses=self.losses)
        gradients_adv = tape.gradient(loss_adv, trainable_vars)
        for v, e_w in zip(trainable_vars, e_w_list):
            v.assign_sub(e_w)

        # optimize
        self.optimizer.apply_gradients(zip(gradients_adv, trainable_vars))

        self.compiled_metrics.update_state(y, y_pred)
        return {m.name: m.result() for m in self.metrics}

In [ ]:
Adam = tf.keras.optimizers.Adam() # Adam lr = 0.0005
cdr = tf.keras.optimizers.schedules.CosineDecayRestarts(initial_learning_rate=0.001,first_decay_steps=50)
cdr = tf.keras.callbacks.LearningRateScheduler(cdr, verbose=1)
# model = Sequential()
AdamW = tfa.optimizers.AdamW(learning_rate=0.001,weight_decay=0.0001, amsgrad=False)
model = Sequential()
model.add(EfficientNetV2S(include_top=False, weights='/content/drive/MyDrive/kw_cv_comp/pretrain/efficientnetv2-s-21k_notop.h5', pooling='avg'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(Dense(26, activation='sigmoid'))
model.compile(optimizer=Adam, loss='binary_crossentropy', metrics='binary_accuracy')
es = EarlyStopping(patience=5, restore_best_weights=True) 
rl = ReduceLROnPlateau(patience=3,verbose=1, min_lr=0.000001) 
checkpoint_filepath_md = '/content/' # 폴드별 모델2 저장 경로 설정('/content/'도 괜찮습니다.)
cb = ModelCheckpoint(filepath=checkpoint_filepath_md + 'effv2M-21k.h5', save_best_only=True, exist_ok=True)
history2= model.fit(train_gen_lab, validation_data=valid_gen_lab, epochs=100, callbacks=[es,cb,rl])

Epoch 1/100
118/118 [==============================] - 114s 717ms/step - loss: 0.7233 - binary_accuracy: 0.5172 - val_loss: 0.6892 - val_binary_accuracy: 0.5366 - lr: 0.0010
Epoch 2/100
118/118 [==============================] - 80s 677ms/step - loss: 0.6925 - binary_accuracy: 0.5391 - val_loss: 0.6929 - val_binary_accuracy: 0.5527 - lr: 0.0010
Epoch 3/100
118/118 [==============================] - 83s 699ms/step - loss: 0.6816 - binary_accuracy: 0.5642 - val_loss: 0.6778 - val_binary_accuracy: 0.5709 - lr: 0.0010
Epoch 4/100
118/118 [==============================] - 83s 702ms/step - loss: 0.6691 - binary_accuracy: 0.5849 - val_loss: 0.6679 - val_binary_accuracy: 0.5857 - lr: 0.0010
Epoch 5/100
118/118 [==============================] - 84s 705ms/step - loss: 0.6551 - binary_accuracy: 0.6066 - val_loss: 0.6537 - val_binary_accuracy: 0.6094 - lr: 0.0010
Epoch 6/100
118/118 [==============================] - 84s 709ms/step - loss: 0.6416 - binary_accuracy: 0.6235 - val_loss: 0.6398 - va

In [ ]:
result = model.evaluate(test_gen_lab)
result

141/141 [==============================] - 23s 161ms/step - loss: 0.5139 - binary_accuracy: 0.7654


[0.5139030814170837, 0.7654103636741638]

In [ ]:
[0.5592584013938904, 0.7297606468200684]

[0.5592584013938904, 0.7297606468200684]

In [ ]:
expect = model.predict(test_gen_lab)
expect

array([[0.15334485, 0.7375551 , 0.45033142, ..., 0.7793036 , 0.06464047,
        0.4812199 ],
       [0.08483452, 0.11866919, 0.02357136, ..., 0.4571069 , 0.95870125,
        0.05330527],
       [0.14395612, 0.04886088, 0.9264191 , ..., 0.9709934 , 0.84366095,
        0.24508762],
       ...,
       [0.17954594, 0.08718614, 0.79394144, ..., 0.980271  , 0.32151937,
        0.01830463],
       [0.44399816, 0.6275713 , 0.06310901, ..., 0.920911  , 0.03742299,
        0.9932614 ],
       [0.41731554, 0.30052707, 0.7823253 , ..., 0.75243133, 0.75602317,
        0.1936917 ]], dtype=float32)

In [ ]:
mark = test_set_data.drop(columns=['label','path'])
for i in range(0,len(mark)) :
    mark.iloc[i,1:] = (expect[i] > 0.5).astype(int) # 확률값이 0.5보다 크다면 1, 작다면 0으로 할당
# mark.drop(columns='index')
dap = test_set_data.drop(columns=['label','path'])

In [ ]:
display(mark,dap)

,index,a,b,c,d,e,f,g,h,i,...,q,r,s,t,u,v,w,x,y,z
8202,8202,0,1,0,0,0,1,1,0,1,...,1,0,0,1,1,1,1,1,0,0
9708,9708,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,1,1,0,1,0
35886,35886,0,0,1,1,1,1,0,0,1,...,1,0,0,1,0,1,0,1,1,0
45881,45881,1,1,0,1,0,1,1,1,0,...,1,0,0,1,0,1,1,1,1,0
21387,21387,1,0,1,1,0,1,1,0,0,...,0,1,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3318,3318,0,0,1,1,0,0,0,0,1,...,0,1,0,0,1,0,0,0,0,0
26375,26375,1,0,1,1,0,1,1,0,0,...,0,1,0,1,0,0,1,0,1,0
44094,44094,0,0,1,0,1,0,0,0,1,...,0,1,0,0,1,1,1,1,0,0
26037,26037,0,1,0,1,0,1,0,0,1,...,1,0,1,0,0,0,0,1,0,1


,index,a,b,c,d,e,f,g,h,i,...,q,r,s,t,u,v,w,x,y,z
8202,8202,0,0,0,0,1,1,0,1,0,...,1,0,1,1,0,0,0,0,1,0
9708,9708,0,1,1,0,0,0,0,1,0,...,1,1,1,0,1,0,0,0,0,1
35886,35886,1,0,0,0,1,1,0,0,1,...,1,0,1,0,0,0,1,1,0,0
45881,45881,1,1,0,1,0,1,1,0,1,...,1,1,1,0,0,0,1,0,0,1
21387,21387,0,0,0,0,1,1,1,0,0,...,1,0,1,0,1,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3318,3318,0,1,0,1,0,0,1,0,0,...,0,1,1,1,0,0,0,0,1,1
26375,26375,0,1,1,1,1,1,0,0,0,...,0,1,1,0,1,0,0,0,1,0
44094,44094,1,0,0,0,0,0,1,0,0,...,0,1,0,1,0,1,0,0,0,0
26037,26037,1,0,0,1,0,0,1,0,1,...,1,1,0,0,1,0,1,1,1,0


In [ ]:
dap3['a'].sum()

NameError: ignored

In [ ]:
def acc_proba(mark, dap, j) :
  check = dap.compare(mark, align_axis=1, keep_equal=True)
  tmp_1 = (check[chr(j)].self + check[chr(j)].other)
  tmp_2 = tmp_1==2
  right_count = tmp_2.sum()
  proba = right_count / dap[chr(j)].sum()

  return proba

In [ ]:
def class_proba(mark, dap, j) :
  check = dap.compare(mark, align_axis=0)
  right_count = check[chr(j)].isnull().sum()/2
  full_count = len(test_set_data)
  proba = right_count / full_count

  return proba

In [ ]:
for j in range(97,123) :
  print('{}의 정답률 : '.format(chr(j)) + str(acc_proba(mark,dap,j)) + '    {}의 클래스 정답률 : '.format(chr(j)) + str(class_proba(mark,dap,j)))